<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 806ms/step - loss: 0.6718 - accuracy: 0.5949 - val_loss: 0.6664 - val_accuracy: 0.5122
Epoch 2/10
26/26 [==============================] - 20s 769ms/step - loss: 0.6190 - accuracy: 0.7117 - val_loss: 0.5923 - val_accuracy: 0.7415
Epoch 3/10
26/26 [==============================] - 20s 777ms/step - loss: 0.5548 - accuracy: 0.7591 - val_loss: 0.5761 - val_accuracy: 0.7220
Epoch 4/10
26/26 [==============================] - 20s 765ms/step - loss: 0.4943 - accuracy: 0.7737 - val_loss: 0.4535 - val_accuracy: 0.7805
Epoch 5/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4654 - accuracy: 0.7895 - val_loss: 0.4283 - val_accuracy: 0.8488
Epoch 6/10
26/26 [==============================] - 20s 762ms/step - loss: 0.3895 - accuracy: 0.8370 - val_loss: 0.3737 - val_accuracy: 0.8780
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3378 - accuracy: 0.8564 - val_loss: 0.2947 - val_accuracy: 0.8927

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.64.h5
26/26 - 21s - loss: 0.6654 - accuracy: 0.5888 - val_loss: 0.6405 - val_accuracy: 0.6244
Epoch 2/5

Epoch 00002: saving model to weights.02-0.65.h5
26/26 - 20s - loss: 0.6199 - accuracy: 0.6715 - val_loss: 0.6502 - val_accuracy: 0.5171
Epoch 3/5

Epoch 00003: saving model to weights.03-0.52.h5
26/26 - 20s - loss: 0.5680 - accuracy: 0.7044 - val_loss: 0.5155 - val_accuracy: 0.7902
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 20s - loss: 0.5069 - accuracy: 0.7603 - val_loss: 0.5274 - val_accuracy: 0.7854
Epoch 5/5

Epoch 00005: saving model to weights.05-0.55.h5
26/26 - 20s - loss: 0.4591 - accuracy: 0.8102 - val_loss: 0.5500 - val_accuracy: 0.7024


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6704 - accuracy: 0.5754 - val_loss: 0.6482 - val_accuracy: 0.7122


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6813 - accuracy: 0.5414 - val_loss: 0.6890 - val_accuracy: 0.4341
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 19s - loss: 0.6605 - accuracy: 0.6083 - val_loss: 0.6637 - val_accuracy: 0.4878


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6586 - accuracy: 0.6314 - val_loss: 0.6269 - val_accuracy: 0.6927
Epoch 2/50
26/26 - 22s - loss: 0.5968 - accuracy: 0.7105 - val_loss: 0.5662 - val_accuracy: 0.7463
Epoch 3/50
26/26 - 20s - loss: 0.5713 - accuracy: 0.7263 - val_loss: 0.5166 - val_accuracy: 0.7756
Epoch 4/50
26/26 - 20s - loss: 0.4942 - accuracy: 0.7944 - val_loss: 0.4573 - val_accuracy: 0.7854
Epoch 5/50
26/26 - 20s - loss: 0.4476 - accuracy: 0.8139 - val_loss: 0.5552 - val_accuracy: 0.6634
Epoch 6/50
26/26 - 20s - loss: 0.3800 - accuracy: 0.8382 - val_loss: 0.4258 - val_accuracy: 0.8293
Epoch 7/50
26/26 - 20s - loss: 0.3373 - accuracy: 0.8625 - val_loss: 0.2768 - val_accuracy: 0.9122
Epoch 8/50
26/26 - 20s - loss: 0.2722 - accuracy: 0.9002 - val_loss: 0.2535 - val_accuracy: 0.9122
Epoch 9/50
26/26 - 21s - loss: 0.2287 - accuracy: 0.9307 - val_loss: 0.2443 - val_accuracy: 0.9171
Epoch 10/50
26/26 - 20s - loss: 0.1676 - accuracy: 0.9574 - val_loss: 0.1655 - val_accuracy: 0.9463
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 801ms/step - loss: 0.6772 - accuracy: 0.5474 - val_loss: 0.6591 - val_accuracy: 0.8000
Epoch 2/5
26/26 [==============================] - 21s 801ms/step - loss: 0.6319 - accuracy: 0.7470 - val_loss: 0.6453 - val_accuracy: 0.5366
Epoch 3/5
26/26 [==============================] - 22s 862ms/step - loss: 0.5733 - accuracy: 0.7616 - val_loss: 0.6296 - val_accuracy: 0.5512
Epoch 4/5
26/26 [==============================] - 23s 869ms/step - loss: 0.4903 - accuracy: 0.8005 - val_loss: 0.4541 - val_accuracy: 0.8585
Epoch 5/5
26/26 [==============================] - 22s 839ms/step - loss: 0.4441 - accuracy: 0.8224 - val_loss: 0.3586 - val_accuracy: 0.8780


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.547445,0.677462,0.800000,0.659132
1,1,0.746959,0.631845,0.536585,0.645303
2,2,0.761557,0.573608,0.551219,0.629637
3,3,0.800487,0.491448,0.858537,0.454133
4,4,0.822384,0.444714,0.878049,0.358627


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 35s 1s/step - loss: 0.6792 - accuracy: 0.5499 - val_loss: 0.6319 - val_accuracy: 0.7415

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 23s 887ms/step - loss: 0.6218 - accuracy: 0.6959 - val_loss: 0.6183 - val_accuracy: 0.7024

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 22s 846ms/step - loss: 0.5996 - accuracy: 0.7384 - val_loss: 0.6067 - val_accuracy: 0.7171

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 23s 892ms/step - loss: 0.5879 - accuracy: 0.7567 - val_loss: 0.6009 - val_accuracy: 0.7171

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 22s 857ms/step - loss: 0.5821

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 25s 952ms/step - loss: 0.6692 - accuracy: 0.5888 - val_loss: 0.6434 - val_accuracy: 0.6537
Epoch 2/50
26/26 [==============================] - 21s 806ms/step - loss: 0.6077 - accuracy: 0.6922 - val_loss: 0.5818 - val_accuracy: 0.7366
Epoch 3/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5640 - accuracy: 0.7138
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 786ms/step - loss: 0.5620 - accuracy: 0.7178 - val_loss: 0.6412 - val_accuracy: 0.5854
Epoch 4/50
26/26 [==============================] - 20s 766ms/step - loss: 0.5133 - accuracy: 0.7555 - val_loss: 0.5252 - val_accuracy: 0.7463
Epoch 5/50
26/26 [==============================] - 20s 776ms/step - loss: 0.4982 - accuracy: 0.7701 - val_loss: 0.5247 - val_accuracy: 0.7805
Epoch 6/50
26/26 [==============================] - 20s 781ms/step - loss: 0.4891 - accuracy: 0.7749 - val_loss: 0.5109 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4870), started 0:17:01 ago. (Use '!kill 4870' to kill it.)